###  Packages importation

In [1]:
from ultralytics import YOLO
import pandas as pd
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
DIR = os.getcwd()
os.chdir("/media/Hcs-screen10-vi/"+DIR.split("/")[3]+"/Felix/Felix/Python")
import Py_Metaxp
from Py_Metaxp import *
os.chdir(DIR)

### Parameters

In [2]:
PID = [33334]  ### Plates ID
SERVER = "NEO-SERVER" ### Server name
#
QUANTILES = [0.0005,0.9995]  ### Min and Max quantiles for images normalization
CONF = 0.3 ### Min Confidence value for objet detection (0 < CONF < 1)
MAX_DET = 5000 ### Maximal number of detected objects by image
IMGSZ = 2048 ### Images size

### Model loading

In [3]:
model_path = "/media/Hcs-screen10-vi/DL_DATA/Felix/Segmentation/Objet_detection/YOLO/TL/04/runs/detect/train5/weights/best.pt"
model = YOLO(model_path)

### Get Images paths

In [177]:
df_path = Py_Get_all_IMPath(SERVER, PID)

def Select_TL(df_path) :
    output_idx = []
    for index, row in df_path.iterrows() :
        if "tl" in row.SOURCE_DESCRIPTION.lower() or "trans" in  row.SOURCE_DESCRIPTION.lower() :
            output_idx.append(index)
    return df_path.iloc[output_idx]
           
df_path = Select_TL(df_path)
print(df_path.shape)
df_path.head()

(7020, 8)


,PLATE_ID,WellID,X_POSITION,Y_POSITION,T_INDEX,Z_INDEX,SOURCE_DESCRIPTION,PATH
0,33334,B02,1.0,1.0,7,1,TL25,/media/Hcs-screen10-vi/MDStorage_18/NEO-STORAG...
1,33334,B02,2.0,1.0,7,1,TL25,/media/Hcs-screen10-vi/MDStorage_18/NEO-STORAG...
2,33334,B02,3.0,1.0,7,1,TL25,/media/Hcs-screen10-vi/MDStorage_18/NEO-STORAG...
3,33334,B02,1.0,2.0,7,1,TL25,/media/Hcs-screen10-vi/MDStorage_18/NEO-STORAG...
4,33334,B02,2.0,2.0,7,1,TL25,/media/Hcs-screen10-vi/MDStorage_18/NEO-STORAG...


In [182]:
60*9*13

7020

### Preprocessing functions

In [178]:
def AutoNormalize(img, q = QUANTILES) :
    quantile = np.quantile(img,q)
    matrix_scaled = (img - quantile[0]) /(quantile[1]-quantile[0])
    matrix_scaled[matrix_scaled <0] = 0
    matrix_scaled[matrix_scaled >1] = 1
    return matrix_scaled

def Img_processing(path) :
    img = io.imread(path)
    img = AutoNormalize(img)
    img = (img*255).astype("uint8")
    img = np.stack([img,img,img], axis = 2)
    return img

### RUN MODEL

In [179]:
df_path = df_path.iloc[0:10000]
df_path.head()

,PLATE_ID,WellID,X_POSITION,Y_POSITION,T_INDEX,Z_INDEX,SOURCE_DESCRIPTION,PATH
0,33334,B02,1.0,1.0,7,1,TL25,/media/Hcs-screen10-vi/MDStorage_18/NEO-STORAG...
1,33334,B02,2.0,1.0,7,1,TL25,/media/Hcs-screen10-vi/MDStorage_18/NEO-STORAG...
2,33334,B02,3.0,1.0,7,1,TL25,/media/Hcs-screen10-vi/MDStorage_18/NEO-STORAG...
3,33334,B02,1.0,2.0,7,1,TL25,/media/Hcs-screen10-vi/MDStorage_18/NEO-STORAG...
4,33334,B02,2.0,2.0,7,1,TL25,/media/Hcs-screen10-vi/MDStorage_18/NEO-STORAG...


In [148]:
def Img_coords(img_path, conf = 0.3) :
    res = model.predict([img_path], conf = conf, max_det = 5000, imgsz = 2048, save = False, line_width = 2, verbose = False, save_txt = False,iou = 0.5, agnostic_nms = True)
    data = pd.DataFrame()
    boxes = res[0].boxes
    data["conf"] = boxes.conf.tolist()
    data["pred_classe"] =  boxes.cls.tolist()
    coords = boxes.xywh.tolist()
    data["cx"] = [c[0] for c in coords]
    data["cy"] = [c[1] for c in coords]
    #data = data[data.conf > 0.5]
    plt.figure(figsize =(15,15))
    #plt.imshow(res[0].plot(font_size = 0.2, conf = True, labels=True, line_width =2, probs = True))
    plt.imshow(res[0].orig_img)
    plt.scatter(data[data.pred_classe == 0].cx, data[data.pred_classe == 0].cy, c="b", s=6)
    plt.scatter(data[data.pred_classe == 1].cx, data[data.pred_classe == 1].cy, c="r",s=6)
    plt.show()
    return data

In [149]:
import time

In [156]:
start = time.time()
df = pd.DataFrame()
for path in df_path.PATH :
    img = Img_processing(path)
    res = model.predict(img, conf = CONF, max_det = MAX_DET, imgsz = IMGSZ, line_width = 2, verbose = False)
    data = pd.DataFrame()
    boxes = res[0].boxes
    data["conf"] = boxes.conf.tolist()
    data["pred_classe"] =  boxes.cls.tolist()
    coords = boxes.xywh.tolist()
    data["cx"] = [c[0] for c in coords]
    data["cy"] = [c[1] for c in coords]
    data_list.append(data)
    
pred_time = time.time()-start
FPS = len(df_path)/ pred_time

df = pd.concat(data_list)
print(df.shape)
print(time.time()-start)
print(FPS)

(350883, 4)
29.820331811904907
6.716706775689888


In [173]:
start = time.time()

path = df_path.PATH[0]
img = io.imread(path)
#img = np.array(Image.open(path))
t1 = time.time()

#matrix = img / 2**16
quantile = np.quantile(img,QUANTILES)
matrix_scaled = (img - quantile[0]) /(quantile[1]-quantile[0])
matrix_scaled[matrix_scaled <0] = 0
matrix_scaled[matrix_scaled >1] = 1
img = matrix_scaled
t2 = time.time()

img = (img*255).astype("uint8")
img = np.stack([img,img,img], axis = 2)
t3 = time.time()

res = model.predict(img, conf = CONF, max_det = MAX_DET, imgsz = IMGSZ, line_width = 2, verbose = False)
t4 = time.time()

data = pd.DataFrame()
boxes = res[0].boxes
data["conf"] = boxes.conf.tolist()
data["pred_classe"] =  boxes.cls.tolist()
coords = boxes.xywh.tolist()
data["cx"] = [c[0] for c in coords]
data["cy"] = [c[1] for c in coords]

print(data.shape)
t5 = time.time()
total = t5-start
print(total)
FPS = 1/ total
print("FPS",FPS)
print("open", (t1-start)/total *100)
print("process", (t2-t1)/total *100)
print("stack", (t3-t2)/total *100)
print("pred", (t4-t3)/total *100)
print("data", (t5-t4)/total *100)

(613, 4)
0.1011052131652832
FPS 9.890686827050507
open 25.95987417053006
process 29.602231728080064
stack 10.95324784349606
pred 29.799842477350225
data 3.6848037805435947


In [168]:
22+40+9+25+3

99

In [101]:
start = time.time()
df = pd.DataFrame()
for path in df_path.PATH :
    img = Img_processing(path)
    res = model.predict(img, conf = CONF, max_det = MAX_DET, imgsz = IMGSZ, line_width = 2, verbose = False)
    data = pd.DataFrame()
    boxes = res[0].boxes
    data["conf"] = boxes.conf.tolist()
    data["pred_classe"] =  boxes.cls.tolist()
    coords = boxes.xywh.tolist()
    data["cx"] = [c[0] for c in coords]
    data["cy"] = [c[1] for c in coords]
    df = pd.concat([df,data])
pred_time = time.time()-start
FPS = len(df_path)/ pred_time
print(df.shape)
print(time.time()-start)
print(FPS)

(111078, 4)
29.455103635787964
6.790047159886558


In [102]:
start = time.time()
data_list = []
for path in df_path.PATH :
    img = Img_processing(path)
    res = model.predict(img, conf = CONF, max_det = MAX_DET, imgsz = IMGSZ, line_width = 2, verbose = False)
    data = pd.DataFrame()
    boxes = res[0].boxes
    data["conf"] = boxes.conf.tolist()
    data["pred_classe"] =  boxes.cls.tolist()
    coords = boxes.xywh.tolist()
    data["cx"] = [c[0] for c in coords]
    data["cy"] = [c[1] for c in coords]
    data_list.append(data)
    
pred_time = time.time()-start
FPS = len(df_path)/ pred_time

df = pd.concat(data_list)
print(df.shape)
print(time.time()-start)
print(FPS)

(111078, 4)
29.720428228378296
6.73169190553861


In [103]:
start = time.time()
data_list = []
for path in df_path.PATH :
    img = Img_processing(path)
    res = model.predict(img, conf = CONF, max_det = MAX_DET, imgsz = 640, line_width = 2, verbose = False)
    data = pd.DataFrame()
    boxes = res[0].boxes
    data["conf"] = boxes.conf.tolist()
    data["pred_classe"] =  boxes.cls.tolist()
    coords = boxes.xywh.tolist()
    data["cx"] = [c[0] for c in coords]
    data["cy"] = [c[1] for c in coords]
    data_list.append(data)
    
pred_time = time.time()-start
FPS = len(df_path)/ pred_time

df = pd.concat(data_list)
print(df.shape)
print(time.time()-start)
print(FPS)

(161, 4)
25.536250114440918
7.836223221638814


In [106]:
start = time.time()
img_list = []
for path in df_path.PATH :
    img = Img_processing(path)
    img_list.append(img)
print(time.time()-start)

23.76786994934082


In [142]:
start = time.time()
img_list = []
for path in df_path.PATH :
    img = np.array(Image.open(path))
    img_list.append(img)
print(time.time()-start)
FPS = len(df_path)/ (time.time()-start)
print(FPS)
print(len(img_list))

37.64897155761719
26.56095287053664
1000


In [ ]:
from skimage import io

In [151]:
start = time.time()
img_list = []
for path in df_path.PATH :
    img = io.imread(path)
    img_list.append(img)
print(time.time()-start)
FPS = len(df_path)/ (time.time()-start)
print(FPS)
print(len(img_list))

4.991281986236572
40.06881516054331
200


In [144]:
img_list[0]

array([[6873, 6820, 6259, ..., 6364, 6531, 6977],
       [7071, 6727, 6377, ..., 6535, 6933, 7468],
       [7213, 6826, 6896, ..., 6347, 6322, 6710],
       ...,
       [8076, 7327, 7105, ..., 7689, 7118, 7271],
       [8038, 7307, 7412, ..., 7344, 7411, 7415],
       [7745, 7891, 7697, ..., 7151, 7120, 7315]], dtype=uint16)

In [121]:
start = time.time()

def Img_processing(path) :
    img = np.array(Image.open(path))
    img = AutoNormalize(img)
    img = (img*255).astype("uint8")
    img = np.stack([img,img,img], axis = 2)
    imgs_liste.append(img)

x  = df_path.PATH
imgs_liste = Manager().list()

if __name__ == '__main__':
    with Pool(multiprocessing.cpu_count()) as pool:
        pool.map(partial(Img_processing),x)
        
print(time.time()-start)
FPS = len(df_path)/ (time.time()-start)
print("FPS",FPS)
print(len(imgs_liste))

5.6428751945495605
FPS 35.441406183325086
200


In [115]:
import cv2

In [117]:
start = time.time()
img_list = []
for path in df_path.PATH :
    img = cv2.imread(path)
    img_list.append(img)
print(time.time()-start)

9.441652774810791


In [112]:
start = time.time()
df = pd.DataFrame()
for img in img_list :
    res = model.predict(img, conf = CONF, max_det = MAX_DET, imgsz = 2048, line_width = 2, verbose = False)
    data = pd.DataFrame()
    boxes = res[0].boxes
    data["conf"] = boxes.conf.tolist()
    data["pred_classe"] =  boxes.cls.tolist()
    coords = boxes.xywh.tolist()
    data["cx"] = [c[0] for c in coords]
    data["cy"] = [c[1] for c in coords]
    #df = pd.concat([df,data])
    
pred_time = time.time()-start
FPS = len(df_path)/ pred_time

print(df.shape)
print(time.time()-start)
print(FPS)

(0, 0)
5.90456223487854
33.87304046367975


In [69]:
import multiprocessing
from multiprocessing import Pool, Process, Manager
from functools import partial

In [135]:
BS = multiprocessing.cpu_count()
print(BS)

32


In [136]:
idx_liste = df_path.index
print(len(idx_liste))
idx_liste

1000


Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            990, 991, 992, 993, 994, 995, 996, 997, 998, 999],
           dtype='int64', length=1000)

In [137]:
def Index_to_Batch(index_liste, BS) :
    output = []
    n = 0
    temp = []
    for i in index_liste :
        n += 1
        if n <= BS :
            temp.append(i)
        else :
            n = 1
            output.append(temp)
            temp = []
            temp.append(i)
    output.append(temp)
    return output

a = Index_to_Batch(idx_liste,BS)

In [138]:
imgs_liste[0].shape

(2048, 2048, 3)

In [139]:
start = time.time()



a =  Index_to_Batch(idx_liste,BS)
data_list = Manager().list()

def Img_processing(x) :
    df_ = df_path.iloc[x]
    img = np.array(Image.open(df_.PATH))
    img = AutoNormalize(img)
    img = (img*255).astype("uint8")
    img = np.stack([img,img,img], axis = 2)
    imgs_liste.append(img)

for x in a :
    imgs_liste = Manager().list()
    
    if __name__ == '__main__':
        with Pool(multiprocessing.cpu_count()) as pool:
            pool.map(partial(Img_processing),x)
    
    for img in imgs_liste :
        res = model.predict(img, conf = CONF, max_det = MAX_DET, imgsz = IMGSZ, line_width = 2, verbose = False)
        data = pd.DataFrame()
        boxes = res[0].boxes
        data["conf"] = boxes.conf.tolist()
        data["pred_classe"] =  boxes.cls.tolist()
        coords = boxes.xywh.tolist()
        data["cx"] = [c[0] for c in coords]
        data["cy"] = [c[1] for c in coords]
        data_list.append(data)

print(len(data_list))
df = pd.concat(data_list)
print(df.shape)    
        
total = time.time()-start
print(total)
FPS = len(df_path)/total
print("FPS",FPS)

1000
(577480, 4)
167.05362391471863
FPS 5.986101807108973


In [180]:
start = time.time()

data_list = Manager().list()

def Img_processing(x) :
    df_ = df_path.iloc[x]
    img = io.imread(df_.PATH)
    img = AutoNormalize(img)
    img = (img*255).astype("uint8")
    img = np.stack([img,img,img], axis = 2)
    imgs_liste.append(img)

x = df_path.index
imgs_liste = Manager().list()

if __name__ == '__main__':
    with Pool(multiprocessing.cpu_count()) as pool:
        pool.map(partial(Img_processing),x)

for img in imgs_liste :
    res = model.predict(img, conf = CONF, max_det = MAX_DET, imgsz = IMGSZ, line_width = 2, verbose = False)
    data = pd.DataFrame()
    boxes = res[0].boxes
    data["conf"] = boxes.conf.tolist()
    data["pred_classe"] =  boxes.cls.tolist()
    coords = boxes.xywh.tolist()
    data["cx"] = [c[0] for c in coords]
    data["cy"] = [c[1] for c in coords]
    data_list.append(data)

    
print(len(data_list))
df = pd.concat(data_list)
print(df.shape)    
        
total = time.time()-start
print(total)
FPS = len(df_path)/total
print("FPS",FPS)

7020
(3827387, 4)
454.93086099624634
FPS 15.430916215767395
